In [ ]:
pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 44.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -q tf-models-official

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -q -U tensorflow-text

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import os

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 

import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#cargamos dataset
df = pd.read_csv("concat_dataset.csv")

In [ ]:
df.head(3)

,ticker_symbol,current_monday,body,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,next_monday,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,abs_dif,rel_dif,type_dif
0,AAPL,2014-12-29,"lx21 made $10,008 on $AAPL -Check it out! htt...",28.4475,28.692499,26.837500,27.332500,24.819241,608353600,2015-01-05,27.072500,28.312500,26.157499,28.002501,25.427635,1.133010e+09,-1.375000,-0.048335,-1
1,AAPL,2015-01-05,Stock Contest!! Pick $AAPL and WIN a FREE Tabl...,27.0725,28.312500,26.157499,28.002501,25.427635,1133010000,2015-01-12,28.150000,28.200001,26.299999,26.497499,24.061016,1.216906e+09,1.077500,0.039801,1
2,AAPL,2015-01-12,#TOPTICKERTWEETS $FB $GOOG $GOOGL $YHOO $SPY $...,28.1500,28.200001,26.299999,26.497499,24.061016,1216906400,2015-01-19,26.959999,28.437500,26.625000,28.245001,25.647839,7.949480e+08,-1.190001,-0.042274,-1


In [ ]:
#quitamos columnas que no usaremos, nos quedamos con texto y numéricas

In [ ]:
df = df[['body', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'type_dif']]

In [ ]:
df.head(2)

,body,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,type_dif
0,"lx21 made $10,008 on $AAPL -Check it out! htt...",28.4475,28.692499,26.837500,27.332500,24.819241,608353600,-1
1,Stock Contest!! Pick $AAPL and WIN a FREE Tabl...,27.0725,28.312500,26.157499,28.002501,25.427635,1133010000,1


In [ ]:
#Split en train y test
from sklearn.model_selection import train_test_split
X = df.drop(columns=["type_dif"])
y = pd.get_dummies(df["type_dif"]).values
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=0)

In [ ]:
X_train_text = X_train.body.values
X_train_nums = X_train.drop(columns=["body"]).values
X_test_text = X_test.body.values
X_test_nums = X_test.drop(columns=["body"]).values

In [ ]:
y_train.shape, X_train_nums.shape

((1053, 3), (1053, 6))

In [ ]:
#aplicamos scaler a las variables numericas
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(X_train_nums)
X_train_nums = scaler.transform(X_train_nums)
X_test_nums = scaler.transform(X_test_nums)

In [ ]:
#cargar BERT

In [ ]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3"
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3"

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
 #arquitectura de la primera parte de la red que trabaja con BERT, incluimos los tokenizer del modelo
 text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
 preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
 encoder_inputs = preprocessing_layer(text_input)
 encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
 outputs = encoder(encoder_inputs)
 net = outputs['pooled_output']
 net = tf.keras.layers.Dropout(0.1)(net)
 net = tf.keras.layers.Dense(10, activation=None, name='classifier')(net)

In [ ]:
#arquitectura conjunto del procesamiento de texto + procesamiento de valores numéricos
N_COLS = 6
inputs = keras.Input(shape=(N_COLS,),name="num_input")
xi = layers.concatenate([inputs,net])
a = layers.Dense(N_COLS*2, activation="relu",name="dense-1.1",kernel_initializer="he_normal")(xi)
a = layers.Dense(N_COLS*2, activation="relu",name="dense-1.2",kernel_initializer="he_normal")(a)
a = layers.Dense(3, activation="softmax",name="output")(a)
#la capa final con activacion softmax ya que se trata de clasificacion

In [ ]:
model = keras.Model(inputs=[inputs,text_input], outputs=[a])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'default': (None, 7 108310273   preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [ ]:
#hiperparametros enfocados en BERT para la arquitectura completa de la red propuesta
epochs = 5
steps_per_epoch = X_train.shape[0]
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy',tf.keras.metrics.AUC()]
)

In [ ]:
model.fit(
    {"text": X_train_text, "num_input": X_train_nums},
    y_train,
    batch_size = 32,
    epochs=5
)

Epoch 1/5
33/33 [==============================] - 852s 26s/step - loss: 0.8622 - accuracy: 0.4425 - auc: 0.7237
Epoch 2/5
33/33 [==============================] - 827s 25s/step - loss: 0.8239 - accuracy: 0.4691 - auc: 0.7385
Epoch 3/5
33/33 [==============================] - 815s 25s/step - loss: 0.8053 - accuracy: 0.5100 - auc: 0.7532
Epoch 4/5
33/33 [==============================] - 815s 25s/step - loss: 0.7486 - accuracy: 0.5926 - auc: 0.8155
Epoch 5/5
33/33 [==============================] - 804s 24s/step - loss: 0.6958 - accuracy: 0.6847 - auc: 0.8702


In [ ]:
evals = model.evaluate(x={'text': X_test_text, 'num_input':X_test_nums}, y=y_test)

17/17 [==============================] - 239s 14s/step - loss: 0.8522 - accuracy: 0.5337 - auc: 0.7572


In [ ]:
evals

[0.8522205948829651, 0.5337187051773071, 0.7571976184844971]

In [ ]:
predictions = model.predict(x={'text': X_test_text, 'num_input':X_test_nums})

In [ ]:
predictions

array([[0.38288528, 0.09401654, 0.52309823],
       [0.2720388 , 0.13837646, 0.5895847 ],
       [0.75745225, 0.02506217, 0.21748558],
       ...,
       [0.454531  , 0.06880014, 0.47666878],
       [0.29935032, 0.13174397, 0.5689058 ],
       [0.53502625, 0.05035448, 0.4146193 ]], dtype=float32)